# ゲート付きRNN

## UGRNN

In [1]:
import torch
import torch.nn as nn

In [2]:
class UGRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

         # 線形変換
        self.transform = nn.Linear(input_size+hidden_size, hidden_size)
        self.update_gate = nn.Linear(input_size+hidden_size, hidden_size)

        # 活性化関数
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input, h_0=None):
        # input（batch firstを想定）: [batch_size, seq_len, input_size]
        batch_size, seq_len, _ = input.size()

        if h_0 is None:
            h_0 = torch.zeros(1, batch_size, self.hidden_size)#.to(device)

        h = h_0.squeeze(0) # [1, batch_size, hidden_size] → [batch_size, hidden_size]
        outputs = []
        for i in range(seq_len):
            # [batch_size, hidden_size]
            combined = torch.cat((input[:, i, :], h), dim = 1)
            h_candidate = self.tanh(self.transform(combined))
            update_gate = self.sigmoid(self.update_gate(combined))
            h = update_gate * h_candidate + (1-update_gate) * h
            outputs.append(h.unsqueeze(1))# [batch_size, hidden_size] -> # [batch_size, 1, hidden_size]
        output_seq = torch.cat(outputs, dim=1)
        h_n = h.unsqueeze(0) # [batch_size, hidden_size] -> [1, batch_size, hidden_size]

        return output_seq, h_n

In [3]:
input_size = 10
hidden_size = 3
seq_len = 4
batch_size = 5

input_tensor = torch.randn(batch_size, seq_len, input_size)
ugrnn = UGRNN(input_size, hidden_size)
output_seq, h_n = ugrnn(input_tensor)

In [5]:
output_seq.shape

torch.Size([5, 4, 3])

## LSTM

In [24]:
class myLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

         # 線形変換
        self.output_gate = nn.Linear(input_size+hidden_size, hidden_size)
        self.forget_gate = nn.Linear(input_size+hidden_size, hidden_size)
        self.update_gate = nn.Linear(input_size+hidden_size, hidden_size)

        # セル状態の更新に必要な全結合層
        self.cell_candidate = nn.Linear(input_size + hidden_size, hidden_size)

        # 活性化関数
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input, h_0=None, c_0 = None):
        # input（batch firstを想定）: [batch_size, seq_len, input_size]
        batch_size, seq_len, _ = input.size()

        if h_0 is None:
            h_0 = torch.zeros(1, batch_size, self.hidden_size)#.to(device)
        if c_0 is None:
            c_0 = torch.zeros(1, batch_size, self.hidden_size)#.to(device)

        h = h_0.squeeze(0) # [1, batch_size, hidden_size] → [batch_size, hidden_size]
        c = c_0.squeeze(0) # [1, batch_size, hidden_size] → [batch_size, hidden_size]
        outputs = []
        for i in range(seq_len):
            # [batch_size, hidden_size]
            combined = torch.cat((input[:, i, :], h), dim = 1) #  [batch_size, input_size]+[batch_size, hidden_size] -> [batch_size, input_size+hidden_size]
            cell_candidate = self.tanh(self.cell_candidate(combined))
            
            update_gate = self.sigmoid(self.update_gate(combined))
            forget_gate = self.sigmoid(self.forget_gate(combined))
            output_gate = self.sigmoid(self.output_gate(combined))
            
            c = update_gate * cell_candidate + forget_gate * c
            h = output_gate * self.tanh(c)
            outputs.append(h.unsqueeze(1)) # [batch_size, hidden_size] -> # [batch_size, 1, hidden_size]
        self.output_seq = torch.cat(outputs, dim=1)
        h_n = h.unsqueeze(0) # [batch_size, hidden_size] -> [1, batch_size, hidden_size]
        c_n = c.unsqueeze(0) # [batch_size, hidden_size] -> [1, batch_size, hidden_size]

        return self.output_seq, (h_n, c_n)

In [26]:
lstm = myLSTM(input_size, hidden_size)
output_seq, (h_n, c_n) = lstm(input_tensor)

In [27]:
output_seq.shape

torch.Size([5, 4, 3])

In [28]:
h_n.shape

torch.Size([1, 5, 3])

In [29]:
c_n.shape

torch.Size([1, 5, 3])

## PytorchのGPUとLSTMを使用する

In [30]:
# LSTM
lstm = nn.LSTM(input_size, hidden_size, batch_first = True)
output_seq, (h_n, c_n) = lstm(input_tensor)

In [31]:
output_seq.shape

torch.Size([5, 4, 3])

In [32]:
h_n.shape

torch.Size([1, 5, 3])

In [33]:
# GRU
gru = nn.GRU(input_size, hidden_size, batch_first = True)
output_seq, h_n = gru(input_tensor)

In [34]:
output_seq.shape

torch.Size([5, 4, 3])

In [35]:
h_n.shape

torch.Size([1, 5, 3])

In [40]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, rnn_type = 'LSTM'):
        super().__init__()
        
        if rnn_type == 'RNN':
            self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)

        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, batch_first = True)

        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size, hidden_size, batch_first = True)

        elif rnn_type == 'UGRNN':
            self.rnn = UGRNN(input_size, hidden_size, batch_first = True)
        else:
            raise ValueError('Unsupported RNN type. Choose from ["LSTM", "RNN", "GRU", "UGRNN"]')

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output_seq, _ = self.rnn(x)
        # output_seq : [batch_size, seq_len, hidden_size]
        # h_n : [1, batch_size, hidden_size]
        out = self.fc(output_seq[:, -1, :])
        return out
        

In [41]:
input_size = 10
hidden_size = 6
seq_len = 4
batch_size = 5
output_size = 3

input_tensor = torch.randn(batch_size, seq_len, input_size)
model = Model(input_size, hidden_size, output_size, rnn_type = 'LSTM')
output = model(input_tensor)

In [42]:
output.shape

torch.Size([5, 3])